In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
df = pd.read_csv('data/wine_data.csv')
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

input_size = 13
output_size = 3
hidden_size = 100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [4]:
model = Net()
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()
EPOCHS = 100
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

Epoch: 10, Loss: 0.9798
Epoch: 20, Loss: 0.7385
Epoch: 30, Loss: 0.6014
Epoch: 40, Loss: 0.5511
Epoch: 50, Loss: 0.5021
Epoch: 60, Loss: 0.4668
Epoch: 70, Loss: 0.4024
Epoch: 80, Loss: 0.3911
Epoch: 90, Loss: 0.2849
Epoch: 100, Loss: 0.2540


In [5]:
torch.save({
    'epoch': epoch, # current epoch
    'model_state_dict': model.state_dict(), # model state
    'optimizer_state_dict': optimizer.state_dict(), # optimizer state
    'loss': loss # last loss
}, 'models/wine_checkpoint.pth')

In [6]:
predict_out = model(Xtest_)
_, predict_y = torch.max(predict_out, 1)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy: {accuracy_score(ytest_, predict_y):.4f}')
print(f'Precision: {precision_score(ytest_, predict_y, average="micro"):.4f}')
print(f'Recall: {recall_score(ytest_, predict_y, average="micro"):.4f}')
print(f'F1 Score: {f1_score(ytest_, predict_y, average="micro"):.4f}')

Accuracy: 0.8611
Precision: 0.8611
Recall: 0.8611
F1 Score: 0.8611
